In [ ]:
from PIL import Image, ImageFilter, ImageOps, ImageDraw, ImageStat
from preprocessor import Preprocessor
import numpy as np

img = Image.open(f'./data_preview/raw_img/ISIC_0000011.jpg')
prep_img = Preprocessor(img)
img

## 1. Reduce dimensions, convert to grayscale

In [ ]:
prep_img.convert_to_grayscale()
prep_img.img.mode

## 2. Contrast enchancement

In [ ]:
prep_img.enchance_contrast()
prep_img.img

## 3. Hair removal

In [ ]:
# im_edges = prep_img.img.filter(ImageFilter.FIND_EDGES)
# blurred = prep_img.img.filter(ImageFilter.GaussianBlur(radius=5))
# prep_img.img.paste(blurred, mask=im_edges)
# prep_img.img
# mask = im_edges.copy()
# #mask = mask.point(lambda x: x > np.average(mask)*5 and 255)
# prep_img.img.paste(im=im_edges, box=(0,0), mask=mask)
# mask

## 4. Vignette effect removal

In [ ]:
prep_img.remove_vignette()
prep_img.img

 ## 5. Resize

In [ ]:
prep_img.resize()
prep_img.img.size

## 6. Add border

In [ ]:
prep_img.add_border()

## Overview and save

In [ ]:
prep_img.save("test_autopreproc.png", path='./src/data_preview/temp/')
prep_img.img

## Leftovers

In [ ]:
source_image = prep_img.img.copy()
#source_image = source_image.crop(box= (20,20, 492, 492) )

height = source_image.size[0]
width = source_image.size[1]
center = (width/2, height/2)

for iteration in range(10):
    print(f"Iteration: {iteration}")
    radius = height / 3 - 20 + iteration * 30

    #Create circular mask
    Y, X = np.ogrid[:width, :height]
    dist_from_center = np.sqrt((X - center[1])**2 + (Y-center[0])**2)
    arr_mask = (dist_from_center <= radius)*255
    
    #Apply mask to source image 
    im_mask = Image.fromarray(arr_mask).convert('L')
    masked_img = Image.composite(source_image, im_mask, mask=im_mask)
    masked_img.show()
    #Get mean pixel values of inner and outer circle
    mean_inner = ImageStat.Stat(source_image, mask=im_mask).mean[0]
    mean_outer = ImageStat.Stat(source_image, mask=ImageOps.invert(im_mask)).mean[0]
    print(f'inner: {mean_inner} \nouter:{mean_outer}\n-------')

    #Check if vignette effect is in the image
    if mean_outer < 25.0:
        inner_fill = np.full((width, height), int(mean_inner))
        inner_fill = Image.fromarray(inner_fill).convert('L')

        corrected_img = Image.composite(source_image, inner_fill,mask=im_mask)
        source_image.paste(corrected_img)
        break 

source_image
